In [1]:
library(dplyr)
library(ggplot2)
library(Seurat)
library(patchwork)
library(processx)
options(warn = -1)
options(repr.plot.width = 20, repr.plot.height = 10, repr.plot.res = 100)

Warning message:
“package ‘dplyr’ was built under R version 4.3.2”

Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Loading required package: SeuratObject

Loading required package: sp


Attaching package: ‘SeuratObject’


The following objects are masked from ‘package:base’:

    intersect, t


Warning message:
“package ‘patchwork’ was built under R version 4.3.2”


In [ ]:
dataset_paths = c('2018838/PRE-01','2018838/PRE-02','2018838/POST-03','2018838/POST-05',
           '2018839/PRE-04','2018839/PRE-05','2018839/POST-06','2018839/POST-11')

dataset_paths <- lapply(dataset_paths, function(x){paste('/tmp/work/Visium/', x, sep = '')})  #constructs the data set pathsseurat_objects <- lapply(dataset_paths, function(x) {Load10X_Spatial(unlist(x))})   #creates a list of seurat objects

seurat_objects <- lapply(dataset_paths, function(x) {Load10X_Spatial(unlist(x))})   #creates a list of seurat objects
sample_names <- lapply(dataset_paths, function(x){tail(unlist(strsplit(unlist(x), split = "/")),n=1)})   #gets the sample name for metadata assignment
seurat_objects <- lapply(seurat_objects, function(x){subset(x,subset = nCount_Spatial > 2000)})

apply_sample_names <- function(x,y){
    x[[]]$sample = y
    return(x)
}
seurat_objects <- mapply(apply_sample_names, seurat_objects, sample_names)  #add sample name information to each seurat object
seurat_objects <- merge(x = seurat_objects[[1]], y = seurat_objects[-1])  #merges object across layers
seurat_objects <- PercentageFeatureSet(seurat_objects, pattern="^MT-", col.name="percent.mt")

saveRDS(seurat_objects,'seurat_objects.rds')

system2("Rscript", args = "/tmp/work/Visium/BRCA_2024/brca_scripts/sctransform.r")

seurat_objects = readRDS('seurat_objects_v2.rds')
seurat_objects <- RunPCA(seurat_objects, assay="SCT")

In [ ]:
seurat_objects <- IntegrateLayers(
  object = seurat_objects, method = HarmonyIntegration,
  orig.reduction = "pca", new.reduction = "harmony",assay = "SCT",
  verbose = FALSE)

seurat_objects <- FindNeighbors(seurat_objects, reduction = "harmony", dims = 1:30)
seurat_objects <- FindClusters(seurat_objects, resolution = 2, cluster.name = "harmony_clusters")

# 
seurat_objects <- RunUMAP(seurat_objects, reduction = "harmony", dims = 1:30, reduction.name = "umap.harmony")

In [ ]:
DimPlot(seurat_objects,reduction = "umap.harmony",group.by = c("sample", "harmony_clusters"),combine = TRUE)

In [ ]:
test <- subset(seurat_objects, subset = sample == 'PRE-01')
SpatialDimPlot(test, label = TRUE, label.size = 3)
SpatialDimPlot(test, cells.highlight = CellsByIdentities(object = test, idents = seq(0, 28)), facet.highlight = TRUE, ncol = 8)

In [ ]:
test <- subset(seurat_objects, subset = sample == 'POST-05')
SpatialDimPlot(test, label = TRUE, label.size = 3)
SpatialDimPlot(test, cells.highlight = CellsByIdentities(object = test, idents = seq(0, 28)), facet.highlight = TRUE, ncol = 8)